In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('Logistic1').getOrCreate()

In [2]:
df = spark.read.csv('FileStore/tables/Interview.csv', inferSchema = True, header = True)

In [3]:
df.columns

In [4]:
from pyspark.sql.functions import lower, col

df = df.withColumn("Client",lower(col('Client name')))
df = df.withColumn("Industry",lower(col('Industry')))
df = df.withColumn("Location",lower(col('Location')))
df = df.withColumn("Interview_Type",lower(col('Interview Type')))
df = df.withColumn("Cand_Loc",lower(col('Candidate Current Location')))
df = df.withColumn("Job_Loc",lower(col('Candidate Job Location')))
df = df.withColumn("Venue",lower(col('Interview Venue')))
df = df.withColumn("Native",lower(col('Candidate Native location')))
df = df.withColumn("Position",lower(col('Position to be closed')))
df = df.withColumn("Skillset",lower(col('Nature of Skillset')))
df = df.withColumn("Marital_Stat",lower(col('Marital Status')))
df = df.withColumn("Expected",lower(col('Expected Attendance')))
df = df.withColumn("ObservedAtt",lower(col('Observed Attendance')))
df = df.withColumn("candidateID",lower(col('Name(Cand ID)')))
df = df.withColumn("Letter_Shared",lower(col('Has the call letter been shared')))
df = df.withColumn("Followup_Call",lower(col('Can I Call you three hours before the interview and follow up on your attendance for the interview')))
df = df.withColumn("Perm_to_start",lower(col('Have you obtained the necessary permission to start at the required time')))
df = df.withColumn("Availability",lower(col('Hope there will be no unscheduled meetings')))

df = df.drop('_c23', '_c24', '_c25', '_c26', '_c27')

In [5]:
from pyspark.sql.functions import regexp_replace
print(df.toPandas()['Client'].unique())

In [6]:
df = df.withColumn('Client', regexp_replace('Client', 'aon hewitt gurgaon', 'aon hewitt'))
df = df.withColumn('Client', regexp_replace('Client', 'aon hewitt', 'hewitt'))
df = df.withColumn('Client', regexp_replace('Client', 'hewitt', 'aon hewitt'))
df = df.withColumn('Client', regexp_replace('Client', 'standard chartered bank chennai', 'standard chartered bank'))
df = df.filter((df['Client']!='\ufeff\ufeff'))
print(df.toPandas()['Client'].unique())

In [7]:
print(df.toPandas()['Industry'].unique())

In [8]:
df = df.withColumn('Industry', regexp_replace('Industry', 'it products and services', 'it services'))
df = df.withColumn('Industry', regexp_replace('Industry', 'it services', 'it'))
print(df.toPandas()['Industry'].unique())

In [9]:
print(df.toPandas()['Location'].unique())

In [10]:
df = df.withColumn('Location', regexp_replace('Location', '- cochin- ', 'cochin'))
df = df.withColumn('Location', regexp_replace('Location', 'gurgaonr', 'gurgaon'))
df = df.withColumn('Location', regexp_replace('Location', 'chennai ', 'chennai'))
print(df.toPandas()['Location'].unique())

In [11]:
print(df.toPandas()['Interview_Type'].unique())

In [12]:
df = df.withColumn('Interview_Type', regexp_replace('Interview_Type', 'sceduled walkin', 'scheduled walk in'))
df = df.withColumn('Interview_Type', regexp_replace('Interview_Type', 'scheduled walk in', 'scheduled walkin'))
df = df.withColumn('Interview_Type', regexp_replace('Interview_Type', 'walkin ', 'walkin'))
df = df.withColumn('Interview_Type', regexp_replace('Interview_Type', 'scheduled ', 'scheduled'))
print(df.toPandas()['Interview_Type'].unique())

In [13]:
print(df.toPandas()['Cand_Loc'].unique())

In [14]:
df = df.withColumn('Cand_Loc', regexp_replace('Cand_Loc', '- cochin- ', 'cochin'))
df = df.withColumn('Cand_Loc', regexp_replace('Cand_Loc', 'chennai ', 'chennai'))
print(df.toPandas()['Cand_Loc'].unique())

In [15]:
print(df.toPandas()['Job_Loc'].unique())

In [16]:
df = df.withColumn('Job_Loc', regexp_replace('Job_Loc', '- cochin- ', 'cochin'))
print(df.toPandas()['Job_Loc'].unique())

In [17]:
print(df.toPandas()['Venue'].unique())

In [18]:
df = df.withColumn('Venue', regexp_replace('Venue', '- cochin- ', 'cochin'))
print(df.toPandas()['Venue'].unique())

In [19]:
print(df.toPandas()['Native'].unique())

In [20]:
df = df.withColumn('Native', regexp_replace('Native', '- cochin- ', 'cochin'))
print(df.toPandas()['Native'].unique())

In [21]:
from pyspark.sql import functions as F

df = df.withColumn( 'New_skillset', F.when(df.Skillset.like("%java%"),"java").when(df.Skillset.like("%oracle%"),"database").when(df.Skillset.like("%sql%"),"database").when(df.Skillset.like("%operations%"),"operations").when(df.Skillset.like("%fresher%"),"fresher").when(df.Skillset.like("%cdd%"),"customer_support").otherwise("other").alias("newSkillset"))

df.select('New_skillset').distinct().show()

In [22]:
df.select('Position').distinct().show()

In [23]:
df = df.withColumn('Position', regexp_replace('Position', 'production- sterile', 'production-sterile'))
df.select('Position').distinct().show()

In [24]:
from pyspark.sql.types import IntegerType
df = df.withColumn('candidateID', regexp_replace('candidateID', 'candidate ',''))
df = df.withColumn('candidateID', df['candidateID'].cast(IntegerType()))

In [25]:
#replace NA with None and drop na
from pyspark.sql.functions import col, when

def blank_as_null(x):
    return when(col(x) != '', col(x)).otherwise(None)

In [26]:
df = df.withColumn('expected', regexp_replace('expected', 'na','')).withColumn('expected', regexp_replace('expected', '11:00 am','yes')).withColumn('expected', regexp_replace('expected', '10.30 am','yes'))
df = df.withColumn('expected', blank_as_null('expected'))
df.select('Expected').distinct().show()

In [27]:
from pyspark.sql.functions import trim
df = df.withColumn('ObservedAtt', trim(df['ObservedAtt']))

In [28]:
df.select('ObservedAtt').distinct().show()

In [29]:
df.columns

In [30]:
df.select('Letter_Shared').distinct().show()

In [31]:
df = df.withColumn('Letter_Shared', regexp_replace('Letter_Shared', 'need to check','uncertain'))
df = df.withColumn('Letter_Shared', regexp_replace('Letter_Shared', 'not yet','uncertain'))
df = df.withColumn('Letter_Shared', regexp_replace('Letter_Shared', 'havent checked','uncertain'))
df = df.withColumn('Letter_Shared', regexp_replace('Letter_Shared', 'not sure','uncertain'))
df = df.withColumn('Letter_Shared', regexp_replace('Letter_Shared', 'yet to check','uncertain'))
df = df.withColumn('Letter_Shared', regexp_replace('Letter_Shared', 'na',''))
df = df.withColumn('Letter_Shared', blank_as_null('Letter_Shared'))
df.select('Letter_Shared').distinct().show()

In [32]:
df.select('Followup_Call').distinct().show()

In [33]:
df = df.withColumn('Followup_Call', regexp_replace('Followup_Call', 'no dont','no'))
df = df.withColumn('Followup_Call', regexp_replace('Followup_Call', 'na',''))
df = df.withColumn('Followup_Call', blank_as_null('Followup_Call'))
df.select('Followup_Call').distinct().show()

In [34]:
df.select('Perm_to_start').distinct().show()

In [35]:
df = df.withColumn('Perm_to_start', regexp_replace('Perm_to_start', 'yet to confirm','uncertain')).withColumn('Perm_to_start', regexp_replace('Perm_to_start', 'not yet','uncertain')).withColumn('Perm_to_start', regexp_replace('Perm_to_start', 'na', ''))
df = df.withColumn('Perm_to_start', blank_as_null('Perm_to_start'))

df.select('Perm_to_start').distinct().show()

In [36]:
df.select('Availability').distinct().show()

In [37]:
df = df.withColumn('Availability', regexp_replace('Availability', 'not sure','uncertain')).withColumn('Availability', regexp_replace('Availability', 'cant say','uncertain')).withColumn('Availability', regexp_replace('Availability', 'na', ''))
df = df.withColumn('Availability', blank_as_null('Availability'))

df.select('Availability').distinct().show()

In [38]:
df_cleaned = df.select('Client','Industry','Location','Position','New_skillset','Interview_Type','Gender','Cand_Loc','Job_Loc','Venue','Native','Marital_Stat','ObservedAtt','Letter_Shared','Followup_Call','Perm_to_start','Availability')

df_cleaned = df_cleaned.na.drop()
df_cleaned.show()

In [39]:
from pyspark.ml.feature import VectorAssembler, VectorIndexer, OneHotEncoder, StringIndexer 


client_indexer = StringIndexer(inputCol = 'Client', outputCol = 'clientIndex')
client_encoder = OneHotEncoder(inputCol = 'clientIndex', outputCol = 'clientVector')

industry_indexer = StringIndexer(inputCol = 'Industry', outputCol = 'industryIndex')
industry_encoder = OneHotEncoder(inputCol = 'industryIndex', outputCol = 'industryVector')

location_indexer = StringIndexer(inputCol = 'Location', outputCol = 'locationIndex')
location_encoder = OneHotEncoder(inputCol = 'locationIndex', outputCol = 'locationVector')

position_indexer = StringIndexer(inputCol = 'Position', outputCol = 'positionIndex')
position_encoder = OneHotEncoder(inputCol = 'positionIndex', outputCol = 'positionVector')

skillset_indexer = StringIndexer(inputCol = 'New_skillset', outputCol = 'skillsetIndex')
skillset_encoder = OneHotEncoder(inputCol = 'skillsetIndex', outputCol = 'skillsetVector')

Interview_Type_indexer = StringIndexer(inputCol = 'Interview_Type', outputCol = 'Interview_TypeIndex')
Interview_Type_encoder = OneHotEncoder(inputCol = 'Interview_TypeIndex', outputCol = 'Interview_TypeVector')

Gender_indexer = StringIndexer(inputCol = 'Gender', outputCol = 'GenderIndex')
Gender_encoder = OneHotEncoder(inputCol = 'GenderIndex', outputCol = 'GenderVector')

Cand_Loc_indexer = StringIndexer(inputCol = 'Cand_Loc', outputCol = 'Cand_LocIndex')
Cand_Loc_encoder = OneHotEncoder(inputCol = 'Cand_LocIndex', outputCol = 'Cand_LocVector')

Job_Loc_indexer = StringIndexer(inputCol = 'Job_Loc', outputCol = 'Job_LocIndex')
Job_Loc_encoder = OneHotEncoder(inputCol = 'Job_LocIndex', outputCol = 'Job_LocVector')

Venue_indexer = StringIndexer(inputCol = 'Venue', outputCol = 'VenueIndex')
Venue_encoder = OneHotEncoder(inputCol = 'VenueIndex', outputCol = 'VenueVector')

Native_indexer = StringIndexer(inputCol = 'Native', outputCol = 'NativeIndex')
Native_encoder = OneHotEncoder(inputCol = 'NativeIndex', outputCol = 'NativeVector')

Marital_Stat_indexer = StringIndexer(inputCol = 'Marital_Stat', outputCol = 'Marital_StatIndex')
Marital_Stat_encoder = OneHotEncoder(inputCol = 'Marital_StatIndex', outputCol = 'Marital_StatVector')

ObservedAtt_indexer = StringIndexer(inputCol = 'ObservedAtt', outputCol = 'ObservedAttIndex')
ObservedAtt_encoder = OneHotEncoder(inputCol = 'ObservedAttIndex', outputCol = 'ObservedAttVector')

Letter_Shared_indexer = StringIndexer(inputCol = 'Letter_Shared', outputCol = 'Letter_SharedIndex')
Letter_Shared_encoder = OneHotEncoder(inputCol = 'Letter_SharedIndex', outputCol = 'Letter_SharedVector')

Followup_Call_indexer = StringIndexer(inputCol = 'Followup_Call', outputCol = 'Followup_CallIndex')
Followup_Call_encoder = OneHotEncoder(inputCol = 'Followup_CallIndex', outputCol = 'Followup_CallVector')

Perm_to_start_indexer = StringIndexer(inputCol = 'Perm_to_start', outputCol = 'Perm_to_startIndex')
Perm_to_start_encoder = OneHotEncoder(inputCol = 'Perm_to_startIndex', outputCol = 'Perm_to_startVector')

Availability_indexer = StringIndexer(inputCol = 'Availability', outputCol = 'AvailabilityIndex')
Availability_encoder = OneHotEncoder(inputCol = 'AvailabilityIndex', outputCol = 'AvailabilityVector')

#assembler = VectorAssembler(inputCols = ['clientVector', 'industryVector', 'locationVector', 'positionVector', 'skillsetVector', 'Interview_TypeVector', 'GenderVector', 'Cand_LocVector', 'Job_LocVector', 'VenueVector', 'NativeVector', 'Marital_StatVector', 'Letter_SharedVector', 'Followup_CallVector', 'Followup_CallVector', 'Perm_to_startVector', 'AvailabilityVector'], outputCol = 'features')

assembler = VectorAssembler(inputCols = ['clientVector', 'industryVector', 'locationVector', 'skillsetVector', 'Interview_TypeVector', 'GenderVector', 'Cand_LocVector', 'Job_LocVector', 'VenueVector', 'NativeVector', 'Marital_StatVector', 'Letter_SharedVector', 'Followup_CallVector', 'Followup_CallVector', 'Perm_to_startVector', 'AvailabilityVector'], outputCol = 'features')


In [40]:
from pyspark.ml import Pipeline

pipeline = Pipeline(stages = [client_indexer, industry_indexer, location_indexer, position_indexer, skillset_indexer, Interview_Type_indexer, Gender_indexer, Cand_Loc_indexer, Job_Loc_indexer, Venue_indexer, Native_indexer, Marital_Stat_indexer, ObservedAtt_indexer, Letter_Shared_indexer, Followup_Call_indexer, Perm_to_start_indexer, Availability_indexer, client_encoder, industry_encoder, location_encoder, position_encoder, skillset_encoder, Interview_Type_encoder, Gender_encoder, Cand_Loc_encoder, Job_Loc_encoder, Venue_encoder, Native_encoder, Marital_Stat_encoder, Letter_Shared_encoder, Followup_Call_encoder, Perm_to_start_encoder, Availability_encoder, assembler])

In [41]:
output = pipeline.fit(df_cleaned).transform(df_cleaned)

In [42]:
final_data =output.select('features','ObservedAttIndex')

In [43]:
train_data, test_data = final_data.randomSplit([0.7,0.3])

In [44]:
from pyspark.ml.feature import ChiSqSelector

selector = ChiSqSelector(featuresCol = 'features',
                         outputCol = 'selectedFeatures', labelCol = "ObservedAttIndex", selectorType = 'fpr') #false positive rate < 0.05 by default

selector_model = selector.fit(train_data)

result = selector_model.transform(train_data)

train_data = result.select('ObservedAttIndex', 'selectedFeatures')
train_data = train_data.withColumnRenamed('selectedFeatures', 'features')

test_data = selector_model.transform(test_data)
test_data = test_data.select('ObservedAttIndex','selectedFeatures')
test_data = test_data.withColumnRenamed('selectedFeatures','features')

In [45]:
#Logistic Regression
from pyspark.ml.classification import LogisticRegression
logistic = LogisticRegression(featuresCol = 'features', labelCol = 'ObservedAttIndex')
fit_model = logistic.fit(train_data)
results = fit_model.transform(test_data)

In [46]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator, MulticlassClassificationEvaluator
my_eval = MulticlassClassificationEvaluator(metricName='accuracy', labelCol='ObservedAttIndex')
my_eval.evaluate(results)

In [47]:
from pyspark.ml.classification import DecisionTreeClassifier, RandomForestClassifier, GBTClassifier
dtc = DecisionTreeClassifier(labelCol = "ObservedAttIndex", featuresCol = "features")
rfc = RandomForestClassifier(numTrees = 100, labelCol = "ObservedAttIndex", featuresCol = "features")
gbc = GBTClassifier(labelCol = "ObservedAttIndex", featuresCol = "features")
  
dtc_model = dtc.fit(train_data)
rfc_model = rfc.fit(train_data)
gbc_model = gbc.fit(train_data)
  
dtc_results = dtc_model.transform(test_data)
rfc_results = rfc_model.transform(test_data)
gbc_results = gbc_model.transform(test_data)
  
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
  
my_eval = MulticlassClassificationEvaluator(metricName= 'accuracy', labelCol='ObservedAttIndex')

In [48]:
my_eval.evaluate(dtc_results)

In [49]:
my_eval.evaluate(rfc_results)

In [50]:
my_eval.evaluate(gbc_results)

In [51]:
from pyspark.sql import *

display(df_cleaned)

In [52]:
df_skillset = df.select('Skillset', 'New_skillset')
df_skillset.groupby(df.Skillset).count().sort(col("count").desc()).show()

In [53]:
display(df_cleaned)

In [54]:
display(df_cleaned)

In [55]:
display(df_cleaned)

In [56]:
display(df_cleaned)

In [57]:
from pyspark.ml.classification import NaiveBayes
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

nb = NaiveBayes(smoothing=1.0, modelType="multinomial", featuresCol = 'features', labelCol = 'ObservedAttIndex')
model = nb.fit(train_data)

nb_results = model.transform(test_data)

# compute accuracy on the test set

my_eval = MulticlassClassificationEvaluator(metricName= 'accuracy', labelCol='ObservedAttIndex')
my_eval.evaluate(nb_results)